In [1]:
from greedy import *

In [9]:
G, latitudes, longitudes = generate_low_degree_g(num_nodes=100)


In [24]:
list(G.out_edges(2))[2][1]


21

In [11]:
nx.get_node_attributes(G, 'x')[0]

5.395781572545246

In [12]:
# num_nodes=100

# G = nx.DiGraph()

# node_range = range(0, num_nodes)
# G.add_nodes_from(node_range)




In [13]:
# latitudes = np.random.uniform(0, 10, size=num_nodes)
# longitudes = np.random.uniform(0, 10, size=num_nodes)

# lat_dict=dict(zip(node_range, latitudes))
# lon_dict=dict(zip(node_range, longitudes))
# nx.set_node_attributes(G, values=lat_dict, name="y")
# nx.set_node_attributes(G, values=lon_dict, name="x")

In [14]:
G.nodes.data()

NodeDataView({0: {'y': 9.650100648652604, 'x': 5.395781572545246}, 1: {'y': 5.583995366186184, 'x': 7.653673677140338}, 2: {'y': 9.09823991173758, 'x': 7.819561183076854}, 3: {'y': 6.165467494536019, 'x': 2.8695316613207256}, 4: {'y': 5.195059724325106, 'x': 7.5961881640905276}, 5: {'y': 3.761229782289642, 'x': 6.250127793807993}, 6: {'y': 5.3000650232516575, 'x': 4.401232227008174}, 7: {'y': 6.32011244811005, 'x': 6.768623407526187}, 8: {'y': 0.5888378852977094, 'x': 4.732708830282118}, 9: {'y': 7.264132143487476, 'x': 2.450466068140491}, 10: {'y': 8.54778906893079, 'x': 5.2834941060307985}, 11: {'y': 8.452137548829386, 'x': 5.462496437879088}, 12: {'y': 7.333690034143709, 'x': 4.650435062800212}, 13: {'y': 7.5433560909134325, 'x': 5.916912075935783}, 14: {'y': 2.6273475504901045, 'x': 3.547884747850958}, 15: {'y': 5.103331644215709, 'x': 5.343056661343737}, 16: {'y': 9.626274718708185, 'x': 2.040174881910457}, 17: {'y': 0.49696853914679484, 'x': 4.370224373031527}, 18: {'y': 3.449181

In [15]:
X, y = generate_dataset_shortest(G, latitudes, latitudes)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42)

batch_size = 20

train_iter = mx.io.NDArrayIter(X, y, batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(X, y, batch_size=batch_size)

logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
model = build_model()
model.bind(data_shapes=train_iter.provide_data,
            label_shapes=train_iter.provide_label)
model.init_params()

model.fit(train_iter,  # train data
            eval_data=val_iter,  # validation data
            optimizer='adam',  # use SGD to train
            #               optimizer_params={'learning_rate':0.01, 'momentum': 0.9},
            eval_metric='acc',  # report accuracy during training
            # output progress for each 100 data batches
            batch_end_callback=mx.callback.Speedometer(batch_size, 10),
            num_epoch=5)  # train for at most 10 dataset passes

/usr/local/lib/python3.8/dist-packages/mxnet/module/base_module.py:501: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  self.init_params(initializer=initializer, arg_params=arg_params, aux_params=aux_params,
INFO:root:Epoch[0] Batch [0-10]	Speed: 6738.43 samples/sec	accuracy=0.413636
INFO:root:Epoch[0] Batch [10-20]	Speed: 5878.66 samples/sec	accuracy=0.405000
INFO:root:Epoch[0] Batch [20-30]	Speed: 5329.42 samples/sec	accuracy=0.355000
INFO:root:Epoch[0] Batch [30-40]	Speed: 5230.23 samples/sec	accuracy=0.395000
INFO:root:Epoch[0] Batch [40-50]	Speed: 4300.84 samples/sec	accuracy=0.400000
INFO:root:Epoch[0] Batch [50-60]	Speed: 4544.28 samples/sec	accuracy=0.420000
INFO:root:Epoch[0] Batch [60-70]	Speed: 6153.89 samples/sec	accuracy=0.365000
INFO:root:Epoch[0] Batch [70-80]	Speed: 5357.19 samples/sec	accuracy=0.485000
INFO:root:Epoch[0] Batch [80-90]	Speed: 5176.75 samples/sec	accuracy=0.425000
INFO:root:Epoch[0] Batch [90-100]	Speed: 6084

In [17]:
src = 1
dst = 2

In [18]:
shortest_path = nx.dijkstra_path(G, src, dst)
print(shortest_path)

NetworkXNoPath: No path to 2.

In [18]:


path = [src]
cur_node = src
total_weights = .0

while True:
    
    if len(path) >= G.number_of_nodes():
        print("FAIL. Path=%s" % path)
        break
        # return path, total_weights, False

    ###
    num_features = 5
    input_vec = np.ones(4 * num_features)

    for idx, out_edge in enumerate(G.out_edges(cur_node)):
        out_edge_dst = out_edge[1]
        input_vec[idx] = G.get_edge_data(cur_node, out_edge[1])['weight']
        input_vec[idx * num_features] = G.get_edge_data(cur_node, out_edge_dst)['weight']
        input_vec[idx * num_features + 1] = latitudes[out_edge_dst]
        input_vec[idx * num_features + 2] = longitudes[out_edge_dst]
        input_vec[idx * num_features + 3] = dist_eucl(
                        longitudes[out_edge_dst], latitudes[out_edge_dst], longitudes[dst], latitudes[dst])
        input_vec[idx * num_features + 4] = dist_cos(longitudes[cur_node], latitudes[cur_node], 
                        longitudes[out_edge_dst], latitudes[out_edge_dst], longitudes[dst], latitudes[dst])

    out_neighbors = list(map(lambda t: (t[1], G.get_edge_data(cur_node, t[1])['weight']),  G.out_edges(cur_node)))

    pred_idx = np.argmax(model.predict(mx.io.NDArrayIter(
        np.array([input_vec]), np.array([0]))).asnumpy()[0])
    next_node, weight = out_neighbors[pred_idx]

    if next_node in path:
        path_set = set(path)
        out_neighbors_set = set(map(lambda n: n[0], out_neighbors))
        available_out_neighbors = out_neighbors_set - path_set
        if len(available_out_neighbors) == 0:
            print("no available neighbors")
            break
        # print(available_out_neighbors)
        random_node = random.choice(list(available_out_neighbors))
        next_node = random_node
        weight = 0 ## TODO fix

    # print("cur_node=%d, pred_idx=%d, next_node=%d, path=%s" % (cur_node, pred_idx, next_node, path))
    ###

    total_weights += weight
    path.append(next_node)

    if next_node == dst:
        print("SUCCESS. Path=%s" % path)
        break
        # return path, total_weights, True

    cur_node = next_node

{72, 9}
SUCCESS. Path=[1, 54, 32, 76, 19, 93, 86, 29, 20, 9, 17, 15, 26, 2]


In [63]:
import osmnx as ox

ox.config(use_cache=True, log_console=True)

# G = ox.graph_from_bbox(-3.68,40.43,-3.67,40.44, network_type="drive")
# G = ox.graph_from_point((-3.68,40.431), dist=100, network_type="drive")

G = ox.graph_from_xml("map.osm")
G_save=G

In [64]:
G=G_save

In [65]:
highway_include = {'motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
    'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link', 'living_street', 'service', 'road'}

In [66]:
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if 'highway' not in meta] 
G.remove_edges_from(edges_to_remove)
# Twice to deal with bidirectional links
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if 'highway' not in meta] 
G.remove_edges_from(edges_to_remove)
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if 'highway' not in meta] 
G.remove_edges_from(edges_to_remove)

In [67]:
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if isinstance(meta['highway'], list) and all(hwy not in highway_include for hwy in meta['highway']) ] 
G.remove_edges_from(edges_to_remove)
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if isinstance(meta['highway'], list) and all(hwy not in highway_include for hwy in meta['highway']) ] 
G.remove_edges_from(edges_to_remove)
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if isinstance(meta['highway'], list) and all(hwy not in highway_include for hwy in meta['highway']) ] 
G.remove_edges_from(edges_to_remove)

In [68]:
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if not isinstance(meta['highway'], list) and meta['highway'] not in highway_include] 
G.remove_edges_from(edges_to_remove)
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if not isinstance(meta['highway'], list) and meta['highway'] not in highway_include] 
G.remove_edges_from(edges_to_remove)
edges_to_remove = [(s,d) for s,d,meta in G.edges.data() if not isinstance(meta['highway'], list) and meta['highway'] not in highway_include] 
G.remove_edges_from(edges_to_remove)

In [69]:
G.remove_edges_from(edges_to_remove)

In [74]:
for s,d,meta in G.edges.data():
    print(s,d,meta['maxspeed'])

KeyError: 'maxspeed'

In [6]:
G = ox.add_edge_speeds(G)
# G = ox.add_edge_travel_times(G)

In [26]:

def add_edge_weights(G):
    n = G.number_of_edges()
    weight_dict = dict(zip(range(n), np.random.uniform(0, 1, size=n)))
    nx.set_edge_attributes(G, 'weight', weight_dict)


def add_xy_to_nodes(G):
    n = G.number_of_nodes()
    lat_dict = dict(zip(range(n), np.random.uniform(0, 10, size=n)))
    lon_dict = dict(zip(range(n), np.random.uniform(0, 10, size=n)))
    nx.set_node_attributes(G, values=lat_dict, name="y")
    nx.set_node_attributes(G, values=lon_dict, name="x")

G = nx.path_graph(5, create_using=nx.DiGraph())
add_xy_to_nodes(G)

In [39]:
weight_dict = {}
for edge in G.edges:
    weight_dict[edge] = np.random.uniform(0, 1)
nx.set_edge_attributes(G, weight_dict, 'weight')

In [40]:
G.edges.data()

OutEdgeDataView([(0, 1, {'weight': 0.76844977937785}), (1, 2, {'weight': 0.9988875790351}), (2, 3, {'weight': 0.7170747144383658}), (3, 4, {'weight': 0.9814882324293592})])

In [41]:
nx.astar_path(G, 0, 4)

[0, 1, 2, 3, 4]

In [51]:
random.choice(list(G.nodes))

2

In [58]:
G.out_edges(1)

OutEdgeDataView([(1, 2)])

In [65]:
list(map(lambda t:  t[1], G.out_edges(2))).index(4)



ValueError: 4 is not in list

In [62]:
m.index(2)

0

In [ ]:
origin_point = (37.792896, -122.412325)
destination_point = (37.790495, -122.408353)
origin_node = ox.get_nearest_node(G, origin_point)
destination_node = ox.get_nearest_node(G, destination_point)
route = nx.shortest_path(G, origin_node, destination_node, weight='length')
fig, ax = ox.plot_graph_route(G, route, origin_point=origin_point, destination_point=destination_point)